In [1]:
import sqlite3
import pandas as pd 
import numpy as np

# SQL commands

In [ ]:
create_tables = [
'''CREATE TABLE IF NOT EXISTS vac (
        vac_id INTEGER PRIMARY KEY,
        vac_name TEXT,
        date TEXT,
        emp_id INTEGER
    );''',
'''CREATE TABLE IF NOT EXISTS emp (
        emp_id INTEGER PRIMARY KEY,
        city TEXT,
        is_trust INTEGER
    );
''',
'''CREATE TABLE IF NOT EXISTS sal (
        vac_id INTEGER PRIMARY KEY,
        salary_from INTEGER,
        salary_to INTEGER,
        gross INTEGER
    )
''']

union_tables = ['''
select * from emp
union
select * from (
		select * from emp_df
		except
		select * from emp) t;
''',

'''
select * from vac
union
select * from (
		select * from vac_df
		except
		select * from vac) t
''',

'''
select * from sal
union
select * from (
		select * from sal_df
		except
		select * from sal) t
''']



--- 

#

In [25]:
DBNAME = 'my.db'

In [26]:
def to_database(sqL_command:str, database_name:str):
    try:
        conn = sqlite3.connect(database_name)
        with conn:
            cur = conn.cursor()
            cur.execute(sqL_command)
            res = cur.fetchall()
            conn.commit()
    except Exception as e:
        print(e)
    conn.close()

In [33]:
vac_df = pd.DataFrame({
    'vac_id':[1856, 1765, 6901, 9301],
    'vac_name':['бизнес аналитик','BI аналитик','BI аналитик','системный аналитик'],
    'date':['2025-01-20','2025-01-21','2025-01-22','2025-01-23'],
    'emp_id':[4,5,5,8]
})

emp_df = pd.DataFrame({
    'emp_id':[4,5,5,8],
    'city':['Kaliningrad','SpB','SpB','Kazan'],
    'is_trust':[1,0,0,1]
})

sal_df = pd.DataFrame({
    'vac_id':[1856, 1765, 6901, 9301],
    'salary_from':[110000, 67900, 85000, 260000],
    'salary_to':[155000, 89900, 95000, 365000],
    'gross':[0,0,1,1]
})

In [35]:
with sqlite3.connect(DBNAME) as conn:
    vac_df.to_sql('vac_df', con=conn, if_exists='replace', index= False)
    emp_df.to_sql('emp_df', con=conn, if_exists='replace', index = False)
    sal_df.to_sql('sal_df', con=conn, if_exists='replace', index = False)
conn.close()

for command in create_tables:
    to_database(command, DBNAME)